<a href="https://colab.research.google.com/github/JunjieLiang-Jack/Audio-Classifier/blob/main/Renzo_AI_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying audio data with convolutional neural networks

### This notebook is inspired by the project "Urban sounds classification with Covnolutional Neural Networks" on "Eduardo Garcia Rajo" Github. 

In [ ]:
#Mount google drive
import sys
from google.colab import drive
drive.mount('/content/drive')     #This version 
!git clone https://github.com/GorillaBus/urban-audio-classifier.git
sys.path.insert(0,'/content/urban-audio-classifier/include')
!pip install xlsxwriter

# Required libraries
import sys
import os
import math
import IPython as IP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pickle
import sys
import helpers
import xlsxwriter
import tensorflow as tf



from IPython.display import clear_output, display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'urban-audio-classifier' already exists and is not an empty directory.


In [ ]:
#The initialsiation of a project is to convert our 24hr audio files to 4sec audio files and then to a common format to ensure we are working with normalized data.  
#The feature extraction techniques, as we are going to work with Mel spectrumgram

Project_number = ''
Project_name= ''
Project_maintaner = ''
Project_startDate = ''    #eg:08232021
Project_folder = ''
Audio_folder = ''

Audio_file_path = '/content/drive/MyDrive/CNN/TestingData/TH358/Audio'
output_Mel = '/content/drive/MyDrive/CNN/TestingData/TH358/Mel_Spec/Mel_Spec'
#output_data = '/content/drive/MyDrive/CNN/TestingData/TH358/extracted_Mel'


## MEL Spectrogram extraction
### Let's extract the MEL Spectrograms to run the prediction 

In [ ]:
#testing data need to extract the Mel_spec inorder to be predicted by the modle. 
features = []
labels = []
counter = 0
n_mels = 40
frames_max = 0
total_test_samples = len(os.listdir(Audio_file_path))

for filename in os.listdir(Audio_file_path):
  if filename.endswith(".wav"):
    file_path = os.path.join(Audio_file_path,filename)
    
    # Extract Log-Mel Spectrograms (do not add padding)
    mels = helpers.get_mel_spectrogram(file_path, 0, n_mels=n_mels)
    
    # Save current frame count
    num_frames = mels.shape[1]

    # Add row (feature / label)
    features.append(mels)
    
    # Update frames maximum
    if (num_frames > frames_max):
        frames_max = 174

    clear_output(wait=True)
    # Notify update every N files
    if (counter == 500):
        print("Status: {}/{}".format(counter+1, total_test_samples))
        counter = 0

    counter += 1
    
print("Finished: {}/{}".format(counter, total_test_samples))

# Add padding to features with less than frames than frames_max
padded_features = helpers.add_padding(features, frames_max)

# Verify shapes
print("Raw features length: {}".format(len(features)))
print("Padded features length: {}".format(len(padded_features)))
print("Feature labels length: {}".format(len(labels)))

# Convert features (X) and labels (y) to Numpy arrays
X = np.array(padded_features)
np.save(output_Mel, X)

#load the Mel_spec
X = np.load('/content/drive/MyDrive/CNN/TestingData/TH358/Mel_Spec/Mel_Spec.npy')

# Check the shape (40,174)
print(X.shape)

Finished: 358/3858
Raw features length: 3858
Padded features length: 3858
Feature labels length: 0


FileNotFoundError: ignored

In [ ]:
#load the Mel_spec
X = np.load('/content/drive/MyDrive/CNN/TestingData/TH358/Mel_Spec/Mel_Spec.npy')

# Check the shape (40,174)
print(X.shape)

(3858, 40, 174)


## Start prediction 

In [ ]:
labels = [
        'Air Conditioner',
        'Car Horn',
        'Children Playing',
        'Dog bark',
        'Drilling',
        'Engine Idling',
        'Gun Shot',
        'Jackhammer',
        'Siren',
        'Street Music'
    ]

# How data should be organized
num_rows = 40
num_columns = 174
num_channels = 1
df= []
Lh = []
i=0

# Reshape to fit the network input (channel last!)
X = X.reshape(X.shape[0], num_rows, num_columns, num_channels)

# Load best saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/CNN/Training Data/UrbanSound8K/models/aug-train-nb3.hdf5')
prediction = model.predict(X,verbose=0)
enumerate_prediction = enumerate(prediction)

for item in enumerate(enumerate_prediction):
  lablepostion = np.argmax(prediction,axis=1) 
  print("Lables: {} \t\t Likelyhood: {}".format(labels[int(np.argmax(prediction[i]))],"{:.2f}".format(item[1][1][lablepostion[i]])))
  label = str(labels[int(np.argmax(prediction[i]))])
  Likelyhood = str(item[1][1][lablepostion[i]])
  Lh.append(Likelyhood)
  df.append(label)
  i = i+1
  #print(np.argmax(prediction,axis=1))
  #print(prediction.index(i)) 
print(Lh)

Lh = np.array(Lh)
#df = pd.DataFrame({'Lable': df},{'Likelyhood': Lh})

# Convert the dataframe to an XlsxWriter Excel object.
#df.to_excel(writer, sheet_name='Prediction results')

workbook = xlsxwriter.Workbook('/content/drive/MyDrive/CNN/TestingData/TH358/result/TH358.xlsx')
worksheet = workbook.add_worksheet()

# Rows and columns are zero indexed.
row = 0
column = 0

# iterating through content list
for item in df :
 
    # write operation perform
    worksheet.write(row, column, item)
 
    # incrementing the value of row by one
    # with each iterations.
    row += 1
column = column +1
row = 0

for score in Lh :
  # write operation perform
  worksheet.write(row, column, score)

  # incrementing the value of row by one
  # with each iterations.
  row += 1
#time calculatio, how far into the audio 

workbook.close()


Lables: Children Playing 		 Likelyhood: 0.84
Lables: Children Playing 		 Likelyhood: 0.75
Lables: Children Playing 		 Likelyhood: 0.42
Lables: Street Music 		 Likelyhood: 0.71
Lables: Children Playing 		 Likelyhood: 0.68
Lables: Street Music 		 Likelyhood: 0.41
Lables: Children Playing 		 Likelyhood: 0.53
Lables: Children Playing 		 Likelyhood: 0.92
Lables: Children Playing 		 Likelyhood: 0.64
Lables: Children Playing 		 Likelyhood: 0.58
Lables: Children Playing 		 Likelyhood: 0.79
Lables: Gun Shot 		 Likelyhood: 0.48
Lables: Children Playing 		 Likelyhood: 0.65
Lables: Children Playing 		 Likelyhood: 0.50
Lables: Children Playing 		 Likelyhood: 0.52
Lables: Children Playing 		 Likelyhood: 0.80
Lables: Children Playing 		 Likelyhood: 0.63
Lables: Engine Idling 		 Likelyhood: 0.45
Lables: Air Conditioner 		 Likelyhood: 0.77
Lables: Air Conditioner 		 Likelyhood: 0.93
Lables: Children Playing 		 Likelyhood: 0.93
Lables: Children Playing 		 Likelyhood: 0.88
Lables: Children Playing 		 Lik

In [ ]:
#file moving 
import shutil
import os
    
source_dir = '/content/drive/MyDrive/CNN/TestingData/TM001_MarsdenPark_Recording/Output'
target_dir = '/content/drive/MyDrive/CNN/TestingData/TM001_MarsdenPark_Recording/Output/Audio'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

In [ ]:
#file delet
import os, shutil
folder = '/content/drive/MyDrive/CNN/TestingData/TM001_MarsdenPark_Recording/Output/Audio'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))